<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Topic Trend Dashboard using ONNXEmbeddings and VectorDistance
 <br>       
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 150px; height: auto; margin-top: 20pt;">
  <br>
    </p>
</header>


<p style = 'font-size:20px;font-family:Arial;'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;'>
Tracking the evolution of topics over time is essential for understanding patterns, behaviors, and emerging trends in large datasets of text. In industries such as customer support, social media monitoring, and market research, identifying how topics shift over time can provide valuable insights for decision-making and strategy development. Traditional manual analysis methods, however, can be labor-intensive and prone to human bias.</p>

<p style = 'font-size:16px;font-family:Arial;'>
In this blog post, we explore a dynamic approach to topic trend analysis by combining message embeddings with topic embeddings, leveraging vector distance calculations to measure similarity between the two. The resulting data will be fed into an interactive dashboard, enabling users to monitor the frequency of topics over specific time periods and set similarity thresholds for enhanced relevance.</p>

<p style = 'font-size:16px;font-family:Arial;'>
While the specific example can be applied across many sectors, we’ll focus on a use case using the Consumer Complaint Database from the Consumer Financial Protection Bureau. This dataset contains complaints about financial products and services, providing valuable insights into consumer sentiment and trends. By categorizing these complaints by topic, businesses can gain a deeper understanding of customer concerns in the consumer finance sector and adjust their strategies to address emerging issues more effectively.</p>

<p style = 'font-size:16px;font-family:Arial;'>To achieve this, we will:   
<ul style = 'font-size:16px;font-family:Arial;'>
  <li>Generate embeddings for both customer messages and inferred/predefined topics</li>
  <li>Calculate vector distances between message and topic embeddings to assess similarity</li>
  <li>Feed the results into a dashboard to display topic trends over time, with configurable similarity thresholds and message counts</li>
   <li>Enable entries of new topics in the dashboard, allowing ad-hoc analyses.</li>
</ul>
<p style = 'font-size:16px;font-family:Arial;'>The approach is visually represented in this diagram:
</p>

<img src=images/workflow_topictrend.png style="border: 4px solid #404040; border-radius: 10px;"/>

<p style = 'font-size:16px;font-family:Arial;'>
This method provides an efficient way to not only categorize messages by topic but also track how these topics evolve over time, offering actionable insights into changing customer concerns, emerging issues, and overall trends.</p>


<p style = 'font-size:20px;font-family:Arial;'><b>Dataset Overview</b></p>
<p style = 'font-size:16px;font-family:Arial;'>
For this analysis, we use the <b>Consumer Complaint Database</b>, a publicly available dataset from the **Consumer Financial Protection Bureau (CFPB)** in the United States. This dataset contains consumer complaints related to financial products and services, helping to identify trends and issues affecting consumers. The CFPB collects and publishes these complaints to promote transparency and consumer protection in the financial industry. The dataset has been used by researchers, regulators, and businesses to analyze market trends, detect fraud, and improve customer service.</p>
<p style = 'font-size:16px;font-family:Arial;'>The full dataset, starting from 2011, contains over 8 million complaints. For this blog post, we apply filters to focus on a manageable subset for analysis. We apply the following filters and obtain around 80k complaints:</p>
<ul style = 'font-size:16px;font-family:Arial;'>
  <li><b>Time Range</b>: Full year 2024</li>
  <li><b>Consent</b>: Complaints where consumers have provided consent to publish their narratives</li>
  <li><b>Geographic Scope</b>: Complaints from the state of California</li>
</ul>



<p style = 'font-size:16px;font-family:Arial;'>The filtered dataset can be accessed here:<a href="https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?consumer_consent_provided=Consent%20provided&date_received_max=2024-12-31&date_received_min=2024-01-01&field=all&format=csv&no_aggs=true&size=80767&state=CA" target="_blank">Download Filtered Dataset (CSV)</a> and is licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License.</p>





| **Field Name**                 | **Description**                                                             |
|--------------------------------|-----------------------------------------------------------------------------|
| `complaint_id`                 | Unique identifier for each complaint.                                       |
| `date_received`                | Date the complaint was received by the CFPB.                               |
| `product`                      | The type of financial product (e.g., mortgage, credit card).               |
| `sub_product`                  | More specific product type (e.g., FHA mortgage, private student loan).     |
| `issue`                        | The issue the consumer is complaining about (e.g., loan modification).     |
| `sub_issue`                    | More detailed issue category.                                              |
| `consumer_complaint_narrative`  | The consumer's description of the complaint.                              |
| `company_public_response`       | The company's public response to the complaint.                            |
| `company`                      | The financial institution involved.                                        |
| `state`                        | The state where the consumer is located.                                   |
| `zip_code`                     | The consumer's ZIP code.                                                   |
| `tags`                         | Tags indicating special characteristics of the complaint (e.g., servicemember). |
| `consumer_consent_provided`     | Indicates if the consumer consented to publish their narrative.           |
| `submitted_via`                | How the complaint was submitted (e.g., web, phone).                       |
| `date_sent_to_company`         | Date the complaint was forwarded to the company.                          |
| `company_response`             | The company's response to the complaint.                                   |
| `timely_response`              | Indicates if the company responded in a timely manner.                    |
| `consumer_disputed`            | Indicates if the consumer disputed the company's response.                 |
| `complaint_what_happened`      | A detailed narrative of the consumer's experience.                        |

<p style = 'font-size:16px;font-family:Arial;'>In the following sections, we will walk through the methodology for embedding generation, similarity analysis, and dashboard visualization.
<p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>1. Connect to Vantage, Import python packages and explore the dataset</b></p>

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
%%capture
!pip install teradataml --upgrade
!pip install teradatasqlalchemy --upgrade
!pip install teradataml-plus --upgrade

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>The above libraries have to be installed. Restart the kernel after executing these cells to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing <b> 0 0</b></i> (zero zero) and pressing <i>Enter</i>.</p>
</div>
<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import tdmlplus
from teradataml import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os, getpass
import plotly.io as pio

In [ ]:
# imports a function `display_dataframes_in_tabs` and `display_wordclouds_in_tabs`
%run utils/tab_widget.py
list_relevant_tables = [] # we will be adding names of relevant tables progressivley into this list to display the tables

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'> 1.1 Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i /home/jovyan/JupyterLabRoot/UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Topictrends_onnxembedding_vectordistance.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:18px;font-family:Arial;'><b>Getting Data for This Demo</b></p>


In [ ]:
%run utils/_dataload.ipynb 
#takes currently 15 minutes (as it's more than 600 MB of data)

In [ ]:
list_relevant_tables.append("consumer_complaints")

In [ ]:
display_dataframes_in_tabs(list_relevant_tables)

<p style = 'font-size:16px;font-family:Arial;'>In addition, we want to check if our database has already got the required functionality to generate embeddings.</p>


In [ ]:
VCL = False # current system is VCE/VCore

In [ ]:
if VCL:
    results = execute_sql("help database mldb").fetchall()
else:
    results = execute_sql("help user mldb").fetchall()

embeddings_functions = [x[0] for x in results if x[0].startswith("ONNXEmbeddings")]
if len(embeddings_functions) >0:#
    print("\n".join(embeddings_functions))
    print("---------------------\nONNXEmbeddings is installed")
else:
    print("ONNXEmbeddings is not installed. Please Upgrade to BYOM version 6")

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>2. Load HuggingFace Model</b>
<p style = 'font-size:16px;font-family:Arial;'>To generate embeddings, we need an ONNX model capable of transforming text into vector representations. We use a pretrained model from
<a href="https://huggingface.co/Teradata/gte-base-en-v1.5" target="_blank">Teradata's Hugging Face repository</a>   , such as gte-base-en-v1.5. The model and its tokenizer are downloaded and stored in Vantage tables as BLOBs using the save_byom function.</p>

In [ ]:
from huggingface_hub import hf_hub_download

model_name = "gte-base-en-v1.5"
number_dimensions_output = 768
model_file_name = "model.onnx" 

In [ ]:
# Step 1: Download Model from Teradata HuggingFace Page
hf_hub_download(repo_id=f"Teradata/{model_name}", filename=f"tokenizer.json", local_dir="./")

In [ ]:
# using the command line syntax as it is more reliable then the python function
!hf download Teradata/{model_name} onnx/{model_file_name} --local-dir ./

In [ ]:
try:
    db_drop_table("embeddings_models")
except:
    pass
try:
    db_drop_table("embeddings_tokenizers")
except:
    pass

In [ ]:
# Step 2: Load Models into Vantage
# a) Embedding model
save_byom(model_id = model_name, # must be unique in the models table
               model_file = f"onnx/{model_file_name}",
               table_name = 'embeddings_models' )
# b) Tokenizer
save_byom(model_id = model_name, # must be unique in the models table
              model_file = 'tokenizer.json',
              table_name = 'embeddings_tokenizers') 

In [ ]:
display_dataframes_in_tabs(["embeddings_models","embeddings_tokenizers"])

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Create the Embeddings</b>

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>3.1 Generate Embeddings with ONNXEmbeddings</b>

<p style = 'font-size:16px;font-family:Arial;'>
Now it's time to generate the embeddings using <b>ONNXEmbeddings</b>.<br>We run the ONNXEmbeddings function to generate embeddings for a small subset of records. The model is <b>loaded into the cache memory on each node</b>, and Teradata's <b>Massively Parallel Processing (MPP)</b> architecture ensures that embeddings are computed in parallel using <b>ONNX Runtime</b> on each node.  <br>Having said that, generating embeddings for the entire training set can be time-consuming, especially when working on a system with limited resources. In the <b>ClearScape Analytics experience</b>, only a <b>4 AMP system</b> with constrained RAM and CPU power is available. To ensure smooth execution, we test embedding generation on a small sample and use <b>pre-calculated embeddings</b> for the remainder of demo. In a real-life scenario you would tyipically encounter multiple hundred AMPs with much more compute power!<br>Also have a look at the most important input parameters of this <b>ONNXEmbeddings</b> function.
<ul style = 'font-size:16px;font-family:Arial;'>
<li><b>InputTable</b>: The source table containing the text to be embedded. </li>
<li><b>ModelTable</b>: The table storing the ONNX model.                    </li>
<li><b>TokenizerTable</b>: The table storing the tokenizer JSON file.       </li>
<li><b>Accumulate</b>: Specifies additional columns to retain in the output </li>  
<li><b>OutputFormat</b>: Specifies the data format of the output embeddings (<b>FLOAT32(768)</b>, matching the model's output dimension).</li>
</ul>
<p style = 'font-size:16px;font-family:Arial;'>
Since embedding generation is computationally expensive, we only process <b>10 records for testing</b> and rely on precomputed embeddings for further analysis.  
</p>


In [ ]:
configure.byom_install_location = "mldb"

In [ ]:
DF_sample10 = DataFrame.from_query("SELECT TOP 10 t.row_id, t.consumer_complaint_narrative as txt FROM consumer_complaints t")

In [ ]:
my_model = DataFrame.from_query(f"select * from embeddings_models where model_id = '{model_name}'")
my_tokenizer = DataFrame.from_query(f"select model as tokenizer from embeddings_tokenizers where model_id = '{model_name}'")

In [ ]:
DF_embeddings_sample = ONNXEmbeddings(
    newdata = DF_sample10,
    modeldata = my_model, 
    tokenizerdata = my_tokenizer, 
    accumulate = ["row_id", "txt"],
    model_output_tensor = "sentence_embedding",
    output_format = f'FLOAT32({number_dimensions_output})',
    enable_memory_check = False
).result

In [ ]:
# using to_pandas() to truncate the txt column. 
DF_embeddings_sample.to_pandas()

The pre-computed embeddings are stored in the table `consumer_embeddings`.

In [ ]:
list_relevant_tables.append("consumer_embeddings")

In [ ]:
DF_embeddings = DataFrame("consumer_embeddings")

In [ ]:
display_dataframes_in_tabs(list_relevant_tables,-1)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Topic Generation</b>
<p style = 'font-size:16px;font-family:Arial;'>When identifying topics from or for textual data, there are generally two approaches:  
<ol style = 'font-size:16px;font-family:Arial;'><li>
    <b>Domain Knowledge-Driven Approach:</b> Topics are predefined based on expert knowledge or business rules.  </li>
    <li><b>Data-Driven Approach:</b>Topics emerge organically from the data itself using unsupervised learning techniques. </li>
    </ol>
<p style = 'font-size:16px;font-family:Arial;'>
    For this analysis, we adopt the <b>data-driven approach</b>, allowing the structure of the dataset to define the topics rather than imposing predefined categories.  For this, we leverage the <b>semantic similarity</b> between text embeddings to group similar complaints. Instead of manually defining topics, we let a clustering algorithm <b>TD_KMEANS</b> discover natural groupings within the data.  
<br>
To ensure manageability, we limit our analysis to 5 clusters. After applying K-Means clustering to the complaint embeddings, we identify the centroids of these clusters, which represent the most central points of each topic group. To understand the nature of each cluster, we extract the 20 distinct complaints closest to each centroid, as these provide the most representative examples of the topic. Instead of manually assigning labels, we leverage a powerful large language model (LLM) to analyze these representative complaints and generate meaningful topic names.</p>

In [ ]:
# this step takes roughly 20 minutes. Note we build a cluster model on >80k rows and >700 dimensions on a small demo system.
# If you want to speed it up, reduce the number of iter_max
num_clusters = 10 # 10 topics
kmeans_out = KMeans(
    id_column="row_id",
    data=DF_embeddings,
    target_columns="emb_0:emb_767",
    output_cluster_assignment=False,
    num_init=10,
    num_clusters=num_clusters,
    iter_max=50,
    seed= 42
)

In [ ]:
print(kmeans_out.show_query())

In [ ]:
copy_to_sql(kmeans_out.model_data, "complaints_clustermodel")

In [ ]:
list_relevant_tables.append("complaints_clustermodel")

In [ ]:
display_dataframes_in_tabs(list_relevant_tables,-1)

In [ ]:
# getting the distance of each message to their cluster centroid. We pick the 20 closest messages
DF_clusterdistance = KMeansPredict(
    data = DF_embeddings,
    object = DataFrame("complaints_clustermodel"),
    output_distance = True   
).result


DF_clusterdistance = DF_clusterdistance.assign(
    rank_distance = DF_clusterdistance.td_distance_kmeans.window(
            partition_columns=DF_clusterdistance.td_clusterid_kmeans,
            order_columns=DF_clusterdistance.td_distance_kmeans
        ).dense_rank()
    )

DF_clusterdistance_top = DF_clusterdistance.loc[DF_clusterdistance.rank_distance<=20]

In [ ]:
DF_complaints = DataFrame('consumer_complaints')

In [ ]:
DF_topmesages = DF_clusterdistance_top.join(
    DF_complaints.select(["row_id","consumer_complaint_narrative"]),
    how = "inner",
    on =  ["row_id = row_id"],
    lsuffix= "a"
).select(["td_clusterid_kmeans", "consumer_complaint_narrative"]).drop_duplicate()
df_topmessages = DF_topmesages.to_pandas()

In [ ]:
df_topmessages

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>5. Visualization</b></p>
<p style = 'font-size:18px;font-family:Arial;'><b>5.1 WordCloud Visualization</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Let's visualize all the clusters through wordcloud visualization.</p>

In [ ]:
wordclouds = []
for i in range(10):
    cluster_feedback = ' '.join(
        df_topmessages[df_topmessages['td_clusterid_kmeans'] == i]['consumer_complaint_narrative'])
    wordclouds.append(WordCloud(width=800, height=400, background_color='white').generate(cluster_feedback))

In [ ]:
#from tab_widget.py
display_wordclouds_in_tabs(wordclouds)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>6. Get Topic Names by asking a LLM</b>
<p style = 'font-size:16px;font-family:Arial;'>
To leverage the summarization capabilities of large-scale language models, we use a multi-billion parameter model to generate meaningful topic names based on representative complaints from each cluster. This step requires an OpenAI API key, as the model runs through an external API. If you don't have an OpenAI API key, use the pre-generated topic names below.<br>Also, feel free to play around with the prompt and see how this changes the cluster names.</p>

In [ ]:
# set to True, if you have an OpenAI key
I_Have_an_OpenAI_API_Key = False

In [ ]:
if I_Have_an_OpenAI_API_Key:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI API KEY")

In [ ]:
if I_Have_an_OpenAI_API_Key:
    prompt_template = """Your task is to identify a common topic of 10 messages that have shown similar vector embeddings. 
    Your answer should be exactly one sentence, maximal 10 words long, summarising the topic. You can skip unneccary filler words.
    The answer should not be starting with "The common topic of the messages is", or "the topic is", or "Customers are complaining" etc.
    
    Here are the 10 messages:
    
    {messages}
    
    ====
    Topic:
    """

In [ ]:
if I_Have_an_OpenAI_API_Key:
    from openai import OpenAI
    results =  {}
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    for i in range (10):
        cluster_feedback = '\n\n'.join(df_topmessages[df_topmessages['td_clusterid_kmeans'] == i]['consumer_complaint_narrative'])
        this_prompt = prompt_template.format(messages = cluster_feedback)
        try:
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": this_prompt,
                    }
                ],
                model="gpt-4o",
                temperature=0,
                max_tokens=4096
            )
            results[i] = chat_completion.choices[0].message.content.strip()
        except Exception as e:
            raise ValueError(f"Failed to call OpenAI API: {str(e)}")
    

In [ ]:
if not I_Have_an_OpenAI_API_Key :
    #pre-defined topics
    results = {
                 0: 'Fraudulent charges and denied claims by banks.',
                 1: 'Violation of consumer privacy rights under Fair Credit Reporting Act.',
                 2: 'Identity theft and fraudulent credit report disputes.',
                 3: 'Disputing inaccurate late payment information on credit reports.',
                 4: 'Disputes over inaccuracies and violations in credit reports.',
                 5: 'Identity theft and removal of fraudulent credit report entries.',
                 6: 'Credit reporting errors and disputes with financial institutions.',
                 7: 'Identity theft and fraudulent accounts affecting credit reports.',
                 8: 'Credit report inaccuracies and data breach concerns.',
                 9: 'Credit report inaccuracies and data breach concerns.'}


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>7. Generate Embeddings for Topics and  get Similarity</b>


<p style = 'font-size:16px;font-family:Arial;'>
Now that we have abstracted topics from the data, we need to generate embeddings for them. This step is crucial because, in the next phase, we will calculate the <b>pairwise similarity</b> between complaints and topics, effectively computing a <b>Cartesian product</b> of all complaint-topic pairs.</p>


In [ ]:
%run utils/topics_distance.py

In [ ]:
try:
    db_drop_table("complaint_topics")
except:
    pass

try:
    db_drop_table("consumer_complaint_topic_similarity")
except:
    pass

In [ ]:
# creates tables complaint_topics (generates embeddings) and consumer_complaint_topic_similarity (calculates cross-wise vector distances), takes 2 minutes
calculate_similarity_topics(results)

In [ ]:
list_relevant_tables.append("complaint_topics") # embeddings for topics
list_relevant_tables.append("consumer_complaint_topic_similarity") # similarity between complaints and topics

In [ ]:
display_dataframes_in_tabs(list_relevant_tables,-2)

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>7.1 Interactive Dashboard for BI reporting</b></p>
<p style = 'font-size:16px;font-family:Arial;'>As a final step, we build a dashboard designed to serve as a <b>business intelligence (BI) reporting tool</b>, allowing us to analyze how topic prevalence changes over time. This interactive dashboard provides a structured way to explore complaint trends and refine topic detection dynamically.
<ul style = 'font-size:16px;font-family:Arial;'>Dashboard Requirements
    <li><b>Visualizing topic trends:</b> Display the number of complaints per topic per month using a <b>multi-line chart</b>, filtering only those complaints with a similarity score above a defined threshold (default: <b>0.6</b>).</li>  
<li><b>Dynamic threshold adjustment:</b> Allow users to modify the similarity threshold, automatically updating the visualization in real time.  
    </li></ul>
<p style = 'font-size:16px;font-family:Arial;'>
The dashboard logic is encapsulated in the <code>`topics_widget.py`</code> module.</p>

In [ ]:
%run utils/topics_widget.py
#to get get_complaints_app()

In [ ]:
pio.renderers.default = "notebook_connected"

In [ ]:
get_complaints_app()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;'>In this demo we have seen that how a <b>fully data-driven approach</b> can help analyze large volumes of text data, automatically identifying topics and tracking their trends over time. Instead of relying on <b>prompt engineering</b> to classify messages—which can be inconsistent, expensive, and hard to scale—we used <b>embeddings, clustering and Vector Distance</b> to get a <b>deterministic and repeatable</b> solution.  <br>
By applying <b>K-Means clustering</b> on complaint embeddings, we discovered topics without predefining them. A <b>large language model (LLM)</b> then helped generate human-readable names for these clusters, but only once—keeping costs low while still benefiting from its summarization power. From there, we converted topic names into embeddings and calculated <b>vector similarities</b>, allowing us to efficiently map messages to topics in a <b>scalable and automated</b> way.<br>The final step was building an <b>interactive BI dashboard</b> that lets users explore topic trends over time and tweak similarity thresholds. <br>
With this approach, we get the <b>best of both worlds</b>: the flexibility of unsupervised learning, the power of embeddings, and the practicality of real-time reporting—all while keeping things <b>scalable, cost-efficient, and environmentally friendly</b>.    
</p> 

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>8. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
# Set to False so you can resume this notebook later without having to load all the data, again.
delete_embeddings = False

if delete_embeddings:
    %run utils/_dataremove.ipynb

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>